In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import Word2Vec
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("dataset.csv")
df.head()

,Unnamed: 0,annotation,GRNTI
0,0,Рассматриваются аспекты и параметры алгоритма ...,50. Автоматика. Вычислительная техника
1,1,Преодолена итерационная процедура решения тран...,73. Транспорт
2,2,Статья посвящена рассмотрению методов управлен...,06. Экономика. Экономические науки
3,3,Впервые используется теорема Геделя о неполнот...,27.43. Математика / Теория вероятностей и мате...
4,4,В 1880-1892 гг. в Якутской области в политичес...,03.61. История. Исторические науки / Этнографи...


In [3]:
# Приведение к нижнему регистру и очистка текста с помощью регулярных выражений
def preprocess(text):
    text = text.lower() 
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 
# Очимстка от стоп-слов
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('russian')]
    return ' '.join(a)

# Лемматизация
wl = WordNetLemmatizer()
 
# Костыль для лемматизации
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Токенизация
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string))
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)]
    return " ".join(a)

In [4]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
df['clean_text'] = df['annotation'].apply(lambda x: finalpreprocess(x))
df.head()

,Unnamed: 0,annotation,GRNTI,clean_text
0,0,Рассматриваются аспекты и параметры алгоритма ...,50. Автоматика. Вычислительная техника,рассматриваются аспекты параметры алгоритма по...
1,1,Преодолена итерационная процедура решения тран...,73. Транспорт,преодолена итерационная процедура решения тран...
2,2,Статья посвящена рассмотрению методов управлен...,06. Экономика. Экономические науки,статья посвящена рассмотрению методов управлен...
3,3,Впервые используется теорема Геделя о неполнот...,27.43. Математика / Теория вероятностей и мате...,впервые используется теорема геделя неполноте ...
4,4,В 1880-1892 гг. в Якутской области в политичес...,03.61. История. Исторические науки / Этнографи...,гг якутской области политической ссылке побыва...


In [5]:
X = df['clean_text']
y = df['GRNTI']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,shuffle=True)

#Word2Vec
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [6]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
            return self
    def transform(self, X):
            return np.array([
                np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                        or [np.zeros(self.dim)], axis=0)
                for words in X
            ])


df['clean_text_tok']=[nltk.word_tokenize(i) for i in df['clean_text']]
model = Word2Vec(df['clean_text_tok'],min_count=1)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))
modelw = MeanEmbeddingVectorizer(w2v)

X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_test_tok)

In [7]:
lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  

y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))

                                                                                                        precision    recall  f1-score   support

                                                                        00. Общественные науки в целом       0.00      0.00      0.00         2
                                                                                         02. Философия       0.00      0.00      0.00         1
                                                                                        27. Математика       0.43      1.00      0.60         6
                                                                                             31. Химия       0.33      1.00      0.50         1
                                                                    31.19. Химия / Аналитическая химия       0.00      0.00      0.00         3
                                                                                          34. Биология       0.75      1.00      0.86  

In [8]:
lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(X_train_vectors_w2v, y_train)

y_predict = lr_w2v.predict(X_val_vectors_w2v)
y_prob = lr_w2v.predict_proba(X_val_vectors_w2v)[:,1]
print(classification_report(y_test,y_predict))

                                                                                                        precision    recall  f1-score   support

                                                                        00. Общественные науки в целом       0.00      0.00      0.00         2
                                                                                         02. Философия       0.00      0.00      0.00         1
                                                                                        27. Математика       0.29      1.00      0.44         6
                                                                                             31. Химия       0.00      0.00      0.00         1
                                                                    31.19. Химия / Аналитическая химия       0.00      0.00      0.00         3
                                                                                          34. Биология       0.00      0.00      0.00  

In [9]:
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  

y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))

                                                                                                        precision    recall  f1-score   support

                                                                        00. Общественные науки в целом       0.00      0.00      0.00         2
                                                                                         02. Философия       0.00      0.00      0.00         1
                                                                                        27. Математика       0.29      1.00      0.44         6
                                                                                             31. Химия       0.00      0.00      0.00         1
                                                                    31.19. Химия / Аналитическая химия       0.00      0.00      0.00         3
                                                                                          34. Биология       0.00      0.00      0.00  

In [11]:
df['clean_text'] = df['annotation'].apply(lambda x: finalpreprocess(x))
X_test=df['clean_text'] 

X_vector=tfidf_vectorizer.transform(X_test)

y_predict = lr_tfidf.predict(X_vector)      
y_prob = lr_tfidf.predict_proba(X_vector)[:,1]
df['predict_prob']= y_prob
df['target']= y_predict
final=df[['clean_text','target']].reset_index(drop=True)
final.head()

,clean_text,target
0,рассматриваются аспекты параметры алгоритма по...,50. Автоматика. Вычислительная техника
1,преодолена итерационная процедура решения тран...,73. Транспорт
2,статья посвящена рассмотрению методов управлен...,06. Экономика. Экономические науки
3,впервые используется теорема геделя неполноте ...,27.43. Математика / Теория вероятностей и мате...
4,гг якутской области политической ссылке побыва...,03.61. История. Исторические науки / Этнографи...


In [12]:
import pickle

# Сохранение модели
pkl_filename = "best_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(lr_tfidf, file)

# Загрузка модели
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)


In [13]:
model

LogisticRegression(C=10, solver='liblinear')